**INSTRUCTIONS**

`1st Video`: run notebook AS-IS.

**After running the notebook for `1st Video` and getting the output video:**

`2nd Video`: comment lines having the comment `1st Video` and uncomment lines having the comment `2nd Video`, rerun the cells and run notebook AS-IS.

**ALL CELLS WHERE CHANGES ARE REQUIRED HAVE A CELL ABOVE IT TO NOTIFY YOU**

In [1]:
import os
import numpy as np
import pylab
import imageio
from matplotlib import pyplot as plt
import cv2
import time
from os.path import isfile, join
from keras.applications import mobilenet
from keras.models import load_model
from scipy.ndimage.measurements import label
from scipy.ndimage.measurements import center_of_mass
from matplotlib import colors
import skimage
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

C:\Users\anshu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'Bebop', 'Bebop2_20180414154341-0700.mp4', 'Bebop2_20180414163256-0700.mp4', 'bebop_mobilenet_overfit_v1.h5', 'bebop_mobilenet_v0.h5', 'Frames', 'Frames_1', 'Frames_2', 'Frames_New', 'InputVideoToHeatMapVideo.ipynb', 'out.mp4', 'out1.mp4', 'Semantic_Segmentation.ipynb', 'Video Frame Extraction.ipynb']


In [3]:
# normalization
# normalize each chip
samplewise_center = True
samplewise_std_normalization = True
# normalize by larger batches
featurewise_center = False
featurewise_std_normalization = False

# adjacent pixel correllation reduction
# never explored
zca_whitening = False
zca_epsilon = 1e-6

# data augmentation 
# training only
transform = 0
zoom_range = 0
color_shift = 0
rotate = 0
flip = False

datagen_test = ImageDataGenerator(
        samplewise_center=samplewise_center,
        featurewise_center=featurewise_center,
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=samplewise_std_normalization,
        zca_whitening=zca_whitening,
        zca_epsilon=zca_epsilon,
        rotation_range=rotate,
        width_shift_range=transform,
        height_shift_range=transform,
        shear_range=transform,
        zoom_range=zoom_range,
        channel_shift_range=color_shift, 
        fill_mode='constant',
        cval=0,
        horizontal_flip=flip,
        vertical_flip=flip,
        rescale=1./255,
        preprocessing_function=None)


In [4]:
#Load Weights
model = load_model('bebop_mobilenet_overfit_v1.h5', custom_objects={
                   'relu6': mobilenet.relu6,
                   'DepthwiseConv2D': mobilenet.DepthwiseConv2D})

In [5]:
def ProcessChip (frame):

    values = np.zeros((9,16,3))
    chips = np.zeros((144,128,128,3))
    
    for i in range(0,9):
        for j in range(0,16):
            chips[16*i+j] = resize(frame[120*i:120*(i+1), 120*j:120*(j+1), :], (128,128,3))
            
    generator_test = datagen_test.flow(
        chips, 
        batch_size=144,
        shuffle=False)
    
    return model.predict_generator(generator_test,
                                  steps = 1)

In [6]:
def heatmap (feature_map, frame):
    
    color_mask = np.zeros((1080,1920,3))
    temp_frame = skimage.img_as_float(frame)
    alpha = 0.6
    for i in range (0,9):
        for j in range (0,16):
            if feature_map[i][j] == 2:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [0, 0, 1] #Blue, House
            elif feature_map[i][j] == 1:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [0, 1, 0] #Green, Concrete
            else:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [1, 0, 0] #Red, Don't Care
    color_mask_hsv = colors.rgb_to_hsv(color_mask)
    frame_hsv = colors.rgb_to_hsv(temp_frame)
    frame_hsv[..., 0] = color_mask_hsv[..., 0]
    frame_hsv[..., 1] = color_mask_hsv[..., 1] * alpha
    frame_masked = colors.hsv_to_rgb(frame_hsv)
    
    return frame_masked

In [7]:
def correct_arr (arr) :
    
    arr = arr + 1
    arr[arr>2] = 0
    
    return arr

**Make changes to next cell while running `2nd Video`.**

In [8]:
def VideoToFrames (vid):
    
    count = 0
    for image in vid.iter_data(): 
        #image: numpy array containing image information
        feature_map = ProcessChip(image)
        arr = heatmap(np.reshape(correct_arr(np.argmax(ProcessChip(image), axis=1)), (9,16)), image)
        cv2.imwrite('./Frames_1/frame%d.jpg'%count, arr*255) #1st Video
        #cv2.imwrite('./Frames_2/frame%d.jpg'%count, arr*255)  #2nd Video
        count += 1
            
    return

In [9]:
def convert_frames_to_video(pathIn,pathOut,fps):
    
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each file
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

**Make changes to next cell while running `2nd Video`.**

In [10]:
filename = './Bebop/Bebop2_20180422171942-0700.mp4' #1st Video
#filename = './Bebop/Bebop2_20180422171508-0700.mp4'  #2nd Video

In [11]:
vid = imageio.get_reader(filename, 'ffmpeg') 

In [12]:
VideoToFrames(vid) #Passing the video to be analyzed frame by frame

C:\Users\anshu\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


**Make changes to next cell while running `2nd Video`.**

In [13]:
convert_frames_to_video('./Frames_1/', 'out_942.mp4', 23.82) #1st Video
#convert_frames_to_video('./Frames_2/', 'out_508.mp4', 23.41) #2nd Video

./Frames_1/frame0.jpg
./Frames_1/frame1.jpg
./Frames_1/frame2.jpg
./Frames_1/frame3.jpg
./Frames_1/frame4.jpg
./Frames_1/frame5.jpg
./Frames_1/frame6.jpg
./Frames_1/frame7.jpg
./Frames_1/frame8.jpg
./Frames_1/frame9.jpg
./Frames_1/frame10.jpg
